In [2]:
import functools
import itertools
import math
import re
import statistics
from functools import reduce
from pprint import pprint
from typing import (
    List,
    Optional,
    Generator,
    NamedTuple,
    Tuple,
    Set,
    Any,
    Dict,
    Union,
    Callable,
    Iterable,
    Sequence,
)

import rich.panel
import sklearn.base
from alive_progress import alive_bar
from numpy.distutils.misc_util import is_sequence
from tqdm import tqdm, trange
from log_symbols import LogSymbols

import overrides as overrides
import pandas as pds
import tabulate as tabulate
from collections import namedtuple
from sklearn import linear_model, metrics, preprocessing, model_selection, svm
from tabulate import tabulate
from pathlib import Path
import pickle
from halo import Halo
import random
import argparse
import sys
from sklearn.experimental import enable_halving_search_cv
from sklearn import neural_network, tree
from sklearn.model_selection import (
    HalvingGridSearchCV,
    RandomizedSearchCV,
    HalvingRandomSearchCV,
)
import scipy.stats as scistats

import rich.progress
import commons
from data_types import PickleOut
from rich.traceback import install as niceTracebacks
from rich.table import Table as rTable

from db_actions import db_actions
import numpy as np

from linRegPred import getSplits, getTransformers


In [3]:
with db_actions.connect() as conn:
    dF = pds.read_sql('SELECT * FROM "runtimeScorePredBase1000"', conn)
# print(dF)
all_x_cols_test = [
        "build-linux-kernel1",
        "fio2",
        "fio3",
        "fio4",
        "fio5",
        "fio6",
        "fio7",
        "fio8",
        "fio9",
        "iperf10",
        "iperf11",
        "iperf12",
        "iperf13",
        "john-the-ripper14",
        "john-the-ripper15",
        "ramspeed16",
        "ramspeed17",
        "ramspeed18",
        "ramspeed19",
        "ramspeed20",
        "ramspeed21",
        "ramspeed22",
        "ramspeed23",
        "ramspeed24",
        "ramspeed25",
        "stream26",
        "stream27",
        "stream28",
        "stream29",
        "pCpu",
        "cpus",
        "rss",
        "vmem",
        "rchar",
        "wchar",
        "syscr",
        "syscw",
]
filtered_x_cols_test = ['john-the-ripper15', 'ramspeed17', 'ramspeed18', 'ramspeed19', 'ramspeed20', 'ramspeed21',
                        'ramspeed23', 'ramspeed24', 'ramspeed25', 'pCpu', 'cpus', 'vmem', 'rchar', 'wchar', 'syscr',
                        'syscw']
x_cols_test = filtered_x_cols_test
y_cols_test = "realtime"

X_test_test: pds.DataFrame = dF[x_cols_test]
y: pds.DataFrame = dF[y_cols_test]

## Deprecated

In [ ]:
t = rTable(title="results")
t.add_column("cv")
t.add_column("unknown")
t.add_column("cv score")
t.add_column("unknown score")
for split in getSplits(dF, 1, x_cols_test, y_cols_test, cvSize=1, unknownSize=1):
    X_train, y_train, X_test, y_test, X_unknown, y_unknown, X_trans, ukwfs, cvwfs = split
    model = tree.DecisionTreeRegressor()
    model.fit(X_train, y_train)
    # print(cvwfs, ukwfs, model.score(X_test, y_test), model.score(X_unknown, y_unknown))
    t.add_row(cvwfs[0], ukwfs[0], str(model.score(X_test, y_test)), str(model.score(X_unknown, y_unknown)))
rich.print(t)

In [ ]:
X_trans = getTransformers(X_test_test, 1)
scale1, poly, scale2 = X_trans
X_scaled = scale2.transform(poly.transform(scale1.transform(X_test_test)))
res2 = list()
for i in range(100):
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X_scaled, y, test_size=0.2)
    model = tree.DecisionTreeRegressor()
    model.fit(X_train, y_train)
    res2.append(model.score(X_test, y_test))

In [ ]:
rich.print(np.average(res2), np.std(res2))

## Current

In [4]:
taskNames = pds.unique(dF.taskName)


def pickCVTasks():
    cvTasks = list(taskNames)
    totalData = len(dF)
    while len(dF.query("taskName in @cvTasks")) > .3 * totalData:
        cvTasks = list()
        remainingTasks = list(taskNames)
        while len(dF.query("taskName in @cvTasks")) < .2 * totalData:
            pick = np.random.choice(remainingTasks)
            remainingTasks.remove(pick)
            cvTasks.append(pick)
    return cvTasks


nodeIds = pds.unique(dF.nodeConfig)


def pickCVNodes():
    cvNodes = list(nodeIds)
    totalData = len(dF)
    while len(dF.query("nodeConfig in @cvNodes")) > .85 * totalData:
        cvNodes = list()
        remainingNodes = list(nodeIds)
        while len(dF.query("nodeConfig in @cvNodes")) < .8 * totalData:
            pick = np.random.choice(remainingNodes)
            remainingNodes.remove(pick)
            cvNodes.append(pick)
    return cvNodes

In [ ]:
def listProd(l):
    return reduce(lambda a, x: a * x, l, 1)

In [ ]:
X_trans = getTransformers(X_test_test, 1)
scale1, poly, scale2 = X_trans
X_scaled = scale2.transform(poly.transform(scale1.transform(X_test_test)))
res2 = list()
done = set()
for i in range(100):
    cvNodes = pickCVNodes()
    #while listProd([i - 164 for i in cvNodes]) in done:
    #    cvNodes = pickCVNodes()
    #done.add(listProd([i - 164 for i in cvNodes]))
    train = dF.query("nodeConfig not in @cvNodes")
    X_train = train[x_cols_test]
    X_train = scale2.transform(poly.transform(scale1.transform(X_train)))
    y_train = train[y_cols_test]
    test = dF.query("nodeConfig in @cvNodes")
    X_test = test[x_cols_test]
    X_test = scale2.transform(poly.transform(scale1.transform(X_test)))
    y_test = test[y_cols_test]
    model = tree.DecisionTreeRegressor()
    #model = linear_model.LinearRegression()
    model.fit(X_train, y_train)
    res2.append((cvNodes, model.score(X_test, y_test)))

In [ ]:
t = rTable(title="results")
t.add_column("cv tasks")
t.add_column("score")
for r in res2[:5]:
    cv, s = r
    print(cv, s)
    #t.add_row(", ".join(list(cv)), str(s))
print("average:", np.average([s for cv, s in res2]), "std:", np.std([s for cv, s in res2]))
#rich.print(t)


In [ ]:
tmp = dict()
seen = dict()
for r in res2:
    cv, s = r
    for c in cv:
        if c not in tmp.keys():
            tmp[c] = 0
            seen[c] = 0
        tmp[c] = tmp[c] + s / len(cv)
        seen[c] = seen[c] + 1
for k in tmp.keys():
    tmp[k] = tmp[k] / seen[k]
t1 = tmp.keys()
t2 = tmp.values()
t = list(zip(t1, t2))
print(sorted(t, key=lambda x: x[1]))
print(np.average(list(t2)), np.std(list(t2)))

## Test n cv nodes

In [5]:

nodeIds = pds.unique(dF.nodeConfig)


def pickNCVNodes(n):
    return np.random.choice(nodeIds, size=n)

In [ ]:
X_trans = getTransformers(X_test_test, 1)
scale1, poly, scale2 = X_trans
X_scaled = scale2.transform(poly.transform(scale1.transform(X_test_test)))
res2 = list()
done = set()
for n in range(1, len(nodeIds)):
    for i in range(min(100, len(nodeIds) ** n)):
        cvNodes = pickNCVNodes(n)
        #while listProd([i - 164 for i in cvNodes]) in done:
        #    cvNodes = pickCVNodes()
        #done.add(listProd([i - 164 for i in cvNodes]))
        train = dF.query("nodeConfig not in @cvNodes")
        X_train = train[x_cols_test]
        X_train = scale2.transform(poly.transform(scale1.transform(X_train)))
        y_train = train[y_cols_test]
        test = dF.query("nodeConfig in @cvNodes")
        X_test = test[x_cols_test]
        X_test = scale2.transform(poly.transform(scale1.transform(X_test)))
        y_test = test[y_cols_test]
        model = tree.DecisionTreeRegressor()
        #model = linear_model.LinearRegression()
        model.fit(X_train, y_train)
        res2.append((n, cvNodes, model.score(X_test, y_test)))
    print(n)

In [ ]:
tmp2 = dict()
for r in res2:
    n, cv, s = r
    if n not in tmp2.keys():
        tmp2[n] = list()
    tmp2[n].append(s)
tmp3 = list()
for k, v in tmp2.items():
    print(k, np.average(v), np.std(v))
    tmp3.append(np.average(v))

In [ ]:
import seaborn as sns

p = sns.lineplot(x=range(1, len(nodeIds)), y=tmp3)

## Test rank accuracy

In [8]:
X_trans = getTransformers(X_test_test, 1)
scale1, poly, scale2 = X_trans
cvNodes = pickCVNodes()
train = dF.query("nodeConfig not in @cvNodes")
X_train = train[x_cols_test]
X_train = scale2.transform(poly.transform(scale1.transform(X_train)))
y_train = train[y_cols_test]
test = dF.query("nodeConfig in @cvNodes")
X_test = test[x_cols_test]
X_test = scale2.transform(poly.transform(scale1.transform(X_test)))
y_test = test[y_cols_test]
model = tree.DecisionTreeRegressor()
model.fit(X_train, y_train)

DecisionTreeRegressor()

In [9]:
model.score(X_test, y_test)

0.8753072180422782

In [10]:
model.score(scale2.transform(poly.transform(scale1.transform(X_test_test))), y)

0.8963717890947415

In [11]:
ranks = dF["rank"]

In [12]:
model.predict(scale2.transform(poly.transform(scale1.transform(X_test_test))))

array([ 3646.,  3648.,  9715., ...,  6000.,  7589., 33096.])

In [13]:
tasks = pds.unique(dF["taskName"])
wfs = pds.unique(dF["wfName"])

In [14]:
with db_actions.connect() as conn:
    test_df = pds.read_sql('SELECT * FROM "averageRuntimeScorePredBase1000"', conn)
# print(test_df)
all_x_cols_test = [
        "build-linux-kernel1",
        "fio2",
        "fio3",
        "fio4",
        "fio5",
        "fio6",
        "fio7",
        "fio8",
        "fio9",
        "iperf10",
        "iperf11",
        "iperf12",
        "iperf13",
        "john-the-ripper14",
        "john-the-ripper15",
        "ramspeed16",
        "ramspeed17",
        "ramspeed18",
        "ramspeed19",
        "ramspeed20",
        "ramspeed21",
        "ramspeed22",
        "ramspeed23",
        "ramspeed24",
        "ramspeed25",
        "stream26",
        "stream27",
        "stream28",
        "stream29",
        "pCpu",
        "cpus",
        "rss",
        "vmem",
        "rchar",
        "wchar",
        "syscr",
        "syscw",
]
filtered_x_cols_test = ['john-the-ripper15', 'ramspeed17', 'ramspeed18', 'ramspeed19', 'ramspeed20', 'ramspeed21',
                        'ramspeed23', 'ramspeed24', 'ramspeed25', 'pCpu', 'cpus', 'vmem', 'rchar', 'wchar', 'syscr',
                        'syscw']
x_cols_test = filtered_x_cols_test
y_cols_test = "realtime"

X_test_test: pds.DataFrame = test_df[x_cols_test]
y_test_test: pds.DataFrame = test_df[y_cols_test]


In [20]:
from scipy.stats import rankdata

res3 = pds.DataFrame(columns=["nodeid", "task", "wf", "realtime", "pred", "predrank", "rank"])
for t, w in test_df[["taskName", "wfName"]].values:
    dummy1 = [t]
    dummy2 = [w]
    q = test_df.query("taskName in @dummy1 and wfName in @dummy2")
    p = model.predict(scale2.transform(poly.transform(scale1.transform(q[x_cols_test]))))
    predranks = rankdata(p, method="min")
    for i, row in enumerate(q.values):
        # print(row)
        nid, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, taskName, wfName, _, _, _, _, _, _, _, _, realtime, rank = tuple(
                row)
        res3 = res3.append(
                {"nodeid": nid, "task": taskName, "wf": wfName, "realtime": realtime, "rank": rank, "pred": p[i],
                 "predrank": predranks[i]},
                ignore_index=True)

In [21]:
res3

,nodeid,task,wf,realtime,pred,predrank,rank
0,165,TRIMGALORE,nfcore/chipseq:1.2.2,15730.266667,18014.0,18,16
1,166,TRIMGALORE,nfcore/chipseq:1.2.2,15871.033333,12049.0,1,17
2,167,TRIMGALORE,nfcore/chipseq:1.2.2,15412.533333,12049.0,1,13
3,168,TRIMGALORE,nfcore/chipseq:1.2.2,19916.533333,19683.0,21,22
4,169,TRIMGALORE,nfcore/chipseq:1.2.2,20376.266667,23964.0,25,25
...,...,...,...,...,...,...,...
57586,189,ApplyBQSR,nfcore/sarek:2.7.1,6785.133333,6277.0,26,27
57587,190,ApplyBQSR,nfcore/sarek:2.7.1,6510.266667,5769.0,15,24
57588,191,ApplyBQSR,nfcore/sarek:2.7.1,3831.833333,5921.0,21,10
57589,192,ApplyBQSR,nfcore/sarek:2.7.1,3818.000000,3532.0,6,9


In [33]:
print(np.average((res3["rank"] - res3["predrank"]) ** 2))

85.81200187529302


In [34]:
print(np.average((res3["realtime"] - res3["pred"]) ** 2))

104354513.6746089
